# Active - Passive AllenNLP SRL BERT

In [19]:
from allennlp_models.pretrained import load_predictor

In [20]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [21]:
from checklist.pred_wrapper import PredictorWrapper

In [22]:
import csv

In [23]:
import logging
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

## Active

In [24]:
srl_predictor = load_predictor('structured-prediction-srl-bert')
output = srl_predictor.predict("Peter cuddled someone.")
output

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'verbs': [{'verb': 'cuddled',
   'description': '[ARG0: Peter] [V: cuddled] [ARG1: someone] .',
   'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'O']}],
 'words': ['Peter', 'cuddled', 'someone', '.']}

In [25]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [26]:
d = ["Peter cuddled someone."]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'cuddled',
     'description': '[ARG0: Peter] [V: cuddled] [ARG1: someone] .',
     'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'O']}],
   'words': ['Peter', 'cuddled', 'someone', '.']}],
 array([1.]))

In [27]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [28]:
def get_arg(pred, arg_target='ARG1'):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [29]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [12]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [30]:
def found_arg1_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_1 = get_arg(pred, arg_target='ARG1')

    if arg_1 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

In [13]:
# initialize editor object
editor = Editor()

# create examples ARG0
t = editor.template("{first_name} cuddled someone.", meta=True,nsamples=50, remove_duplicates=True)
with open('/datasets/active_arg0_cuddled_bert.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/active_arg0_cuddled_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 50 examples
Test cases:      50
Fails (rate):    0 (0.0%)
['David', 'cuddled', 'someone', '.'] [ True]
['Leslie', 'cuddled', 'someone', '.'] [ True]
['Ken', 'cuddled', 'someone', '.'] [ True]
['Daniel', 'cuddled', 'someone', '.'] [ True]
['Martin', 'cuddled', 'someone', '.'] [ True]
['Stephen', 'cuddled', 'someone', '.'] [ True]
['Kim', 'cuddled', 'someone', '.'] [ True]
['Lucy', 'cuddled', 'someone', '.'] [ True]
['Henry', 'cuddled', 'someone', '.'] [ True]
['Edith', 'cuddled', 'someone', '.'] [ True]
['Christine', 'cuddled', 'someone', '.'] [ True]
['Joseph', 'cuddled', 'someone', '.'] [ True]
['Ben', 'cuddled', 'someone', '.'] [ True]
['Karen', 'cuddled', 'someone', '.'] [ True]
['Fiona', 'cuddled', 'someone', '.'] [ True]
['Lauren', 'cuddled', 'someone', '.'] [ True]
['Dave', 'cuddled', 'someone', '.'] [ True]
['Andrew', 'cuddled', 'someone', '.'] [ True]
['Matthew', 'cuddled', 'someone', '.'] [ True]
['Jay', 'cuddled', 'someone', '.'] [ True]
['Lynn', 'cuddled', 'someon

In [31]:
# initialize editor object
editor = Editor()

# create examples ARG0
t = editor.template("Peter cuddled {first_name}.", meta=True,nsamples=50, remove_duplicates=True)
with open('/datasets/active_arg1_cuddled_bert.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg1_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/active_arg1_cuddled_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 50 examples
Test cases:      50
Fails (rate):    0 (0.0%)
['Peter', 'cuddled', 'Heather', '.'] [ True]
['Peter', 'cuddled', 'Marie', '.'] [ True]
['Peter', 'cuddled', 'Walter', '.'] [ True]
['Peter', 'cuddled', 'Grace', '.'] [ True]
['Peter', 'cuddled', 'Albert', '.'] [ True]
['Peter', 'cuddled', 'Ron', '.'] [ True]
['Peter', 'cuddled', 'Matt', '.'] [ True]
['Peter', 'cuddled', 'Francis', '.'] [ True]
['Peter', 'cuddled', 'John', '.'] [ True]
['Peter', 'cuddled', 'James', '.'] [ True]
['Peter', 'cuddled', 'Ralph', '.'] [ True]
['Peter', 'cuddled', 'Annie', '.'] [ True]
['Peter', 'cuddled', 'Caroline', '.'] [ True]
['Peter', 'cuddled', 'Kenneth', '.'] [ True]
['Peter', 'cuddled', 'Judith', '.'] [ True]
['Peter', 'cuddled', 'Larry', '.'] [ True]
['Peter', 'cuddled', 'Ken', '.'] [ True]
['Peter', 'cuddled', 'Dorothy', '.'] [ True]
['Peter', 'cuddled', 'Gary', '.'] [ True]
['Peter', 'cuddled', 'Ian', '.'] [ True]
['Peter', 'cuddled', 'Susan', '.'] [ True]
['Peter', 'cuddled', 'B

In [14]:
# initialize editor object
editor = Editor()

# create examples ARG0
t = editor.template("{first_name} kissed someone.", meta=True,nsamples=50, remove_duplicates=True)

with open('/datasets/active_arg0_kissed_bert.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/active_arg0_kissed_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 50 examples
Test cases:      50
Fails (rate):    0 (0.0%)
['Anna', 'kissed', 'someone', '.'] [ True]
['Alexander', 'kissed', 'someone', '.'] [ True]
['Roy', 'kissed', 'someone', '.'] [ True]
['Barbara', 'kissed', 'someone', '.'] [ True]
['Roy', 'kissed', 'someone', '.'] [ True]
['Laura', 'kissed', 'someone', '.'] [ True]
['Lawrence', 'kissed', 'someone', '.'] [ True]
['Anne', 'kissed', 'someone', '.'] [ True]
['Betty', 'kissed', 'someone', '.'] [ True]
['Kelly', 'kissed', 'someone', '.'] [ True]
['Christopher', 'kissed', 'someone', '.'] [ True]
['Ken', 'kissed', 'someone', '.'] [ True]
['Kathleen', 'kissed', 'someone', '.'] [ True]
['Louise', 'kissed', 'someone', '.'] [ True]
['Wendy', 'kissed', 'someone', '.'] [ True]
['Samuel', 'kissed', 'someone', '.'] [ True]
['Lucy', 'kissed', 'someone', '.'] [ True]
['Janet', 'kissed', 'someone', '.'] [ True]
['Adam', 'kissed', 'someone', '.'] [ True]
['Wendy', 'kissed', 'someone', '.'] [ True]
['Jeff', 'kissed', 'someone', '.'] [ True

In [15]:
# initialize editor object
editor = Editor()

# create examples ARG0
t = editor.template("{first_name} does not cuddle someone.", meta=True,nsamples=50, remove_duplicates=True)

with open('/datasets/active_arg0_negation1_bert.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/active_arg0_negation1_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 50 examples
Test cases:      50
Fails (rate):    50 (100.0%)

Example fails:
Don [V: does] not cuddle someone .
----
Michael [V: does] not cuddle someone .
----
Steven [V: does] not cuddle someone .
----
['Charles', 'does', 'not', 'cuddle', 'someone', '.'] [False]
['Al', 'does', 'not', 'cuddle', 'someone', '.'] [False]
['Margaret', 'does', 'not', 'cuddle', 'someone', '.'] [False]
['Ed', 'does', 'not', 'cuddle', 'someone', '.'] [False]
['Alfred', 'does', 'not', 'cuddle', 'someone', '.'] [False]
['Kenneth', 'does', 'not', 'cuddle', 'someone', '.'] [False]
['Diana', 'does', 'not', 'cuddle', 'someone', '.'] [False]
['Cynthia', 'does', 'not', 'cuddle', 'someone', '.'] [False]
['Howard', 'does', 'not', 'cuddle', 'someone', '.'] [False]
['Ed', 'does', 'not', 'cuddle', 'someone', '.'] [False]
['Joan', 'does', 'not', 'cuddle', 'someone', '.'] [False]
['Anthony', 'does', 'not', 'cuddle', 'someone', '.'] [False]
['Jason', 'does', 'not', 'cuddle', 'someone', '.'] [False]
['Peter', 'does

In [16]:
# initialize editor object
editor = Editor()

# create examples ARG0
t = editor.template("{first_name} doesn't cuddle someone.", meta=True,nsamples=50, remove_duplicates=True)

with open('/datasets/active_arg0_negation1.2_bert.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/active_arg0_negation1.2_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 50 examples
Test cases:      50
Fails (rate):    50 (100.0%)

Example fails:
Florence [V: does] n't cuddle someone .
----
Jimmy [V: does] n't cuddle someone .
----
Sandra [V: does] n't cuddle someone .
----
['Jay', 'does', "n't", 'cuddle', 'someone', '.'] [False]
['Amanda', 'does', "n't", 'cuddle', 'someone', '.'] [False]
['Ann', 'does', "n't", 'cuddle', 'someone', '.'] [False]
['Alice', 'does', "n't", 'cuddle', 'someone', '.'] [False]
['Louise', 'does', "n't", 'cuddle', 'someone', '.'] [False]
['Karen', 'does', "n't", 'cuddle', 'someone', '.'] [False]
['Louise', 'does', "n't", 'cuddle', 'someone', '.'] [False]
['Lynn', 'does', "n't", 'cuddle', 'someone', '.'] [False]
['Andrew', 'does', "n't", 'cuddle', 'someone', '.'] [False]
['Colin', 'does', "n't", 'cuddle', 'someone', '.'] [False]
['Larry', 'does', "n't", 'cuddle', 'someone', '.'] [False]
['Lisa', 'does', "n't", 'cuddle', 'someone', '.'] [False]
['Harry', 'does', "n't", 'cuddle', 'someone', '.'] [False]
['Brian', 'does',

In [17]:
# initialize editor object
editor = Editor()

# create examples ARG0
t = editor.template("{first_name} cuddled no one.", meta=True,nsamples=50, remove_duplicates=True)

with open('/datasets/active_arg0_negation2_bert.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/active_arg0_negation2_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 50 examples
Test cases:      50
Fails (rate):    0 (0.0%)
['Michael', 'cuddled', 'no', 'one', '.'] [ True]
['Virginia', 'cuddled', 'no', 'one', '.'] [ True]
['Judy', 'cuddled', 'no', 'one', '.'] [ True]
['Amy', 'cuddled', 'no', 'one', '.'] [ True]
['Hugh', 'cuddled', 'no', 'one', '.'] [ True]
['Richard', 'cuddled', 'no', 'one', '.'] [ True]
['Elizabeth', 'cuddled', 'no', 'one', '.'] [ True]
['Ruth', 'cuddled', 'no', 'one', '.'] [ True]
['Kelly', 'cuddled', 'no', 'one', '.'] [ True]
['Steve', 'cuddled', 'no', 'one', '.'] [ True]
['Larry', 'cuddled', 'no', 'one', '.'] [ True]
['Kevin', 'cuddled', 'no', 'one', '.'] [ True]
['Laura', 'cuddled', 'no', 'one', '.'] [ True]
['Nick', 'cuddled', 'no', 'one', '.'] [ True]
['Hugh', 'cuddled', 'no', 'one', '.'] [ True]
['Mike', 'cuddled', 'no', 'one', '.'] [ True]
['Michelle', 'cuddled', 'no', 'one', '.'] [ True]
['Kate', 'cuddled', 'no', 'one', '.'] [ True]
['Eleanor', 'cuddled', 'no', 'one', '.'] [ True]
['Wendy', 'cuddled', 'no', 'one

# End of Notebook